In [15]:
# This notebook runs separate from the master notebook to be able to generate a data set out of the 
# Bikeshare API which has data different from the historical bikeshare data
# The realtime information provides data on the status of the bike stations
# This can then be aggregated to look at the history of bike stations and gain insights
import json
import pandas as pd
import datetime
from pprint import pprint
import time

In [16]:
#Create a series of lists to populate from the BikeShare API
address = []
term_id = []
lat = []
lng = []
num_bikes = []
num_empty_docks = []
obj_id = []
term_num = []
time_stamp = []

biker_data = {"time":time_stamp,"term_id": term_id, "address": address, "lat": lat, "long": lng, "num_bikes":num_bikes,"num_empty_docks":num_empty_docks, "obj_id":obj_id,"term_num":term_num}
biker_data = pd.DataFrame(biker_data)

with open('bike_data.csv', 'w') as f:
    biker_data.to_csv(f, header=True,index=False)

In [17]:
#This function makes a call to the bikeshare API and saves the desired data fields to the previously created lists
#The lists are then made into a data frame that is appended to a .csv without the headers
#This was done as a function so the .csv could capture continuously streaming data to make a historical data set
def BikerData():

    url = "https://maps2.dcgis.dc.gov/dcgis/rest/services/DCGIS_DATA/Transportation_WebMercator/MapServer/5/query?where=1%3D1&outFields=OBJECTID,ID,ADDRESS,TERMINAL_NUMBER,LATITUDE,LONGITUDE,NUMBER_OF_BIKES,NUMBER_OF_EMPTY_DOCKS&outSR=4326&f=json"

    response = requests.get(url)
    data = response.json()
 
    now = datetime.datetime.now()
    time = pd.Timestamp(now.year,now.month,now.day,now.hour,now.minute)
    for index,item in enumerate(data["features"]):
        address.append(data["features"][index]["attributes"]["ADDRESS"])
        term_id.append(data["features"][index]["attributes"]["ID"])
        lat.append(data["features"][index]["attributes"]["LATITUDE"])
        lng.append(data["features"][index]["attributes"]["LONGITUDE"])
        num_bikes.append(data["features"][index]["attributes"]["NUMBER_OF_BIKES"])
        num_empty_docks.append(data["features"][index]["attributes"]["NUMBER_OF_EMPTY_DOCKS"])
        obj_id.append(data["features"][index]["attributes"]["OBJECTID"])
        term_num.append(data["features"][index]["attributes"]["TERMINAL_NUMBER"])
        time_stamp.append(time)

    biker_data = {"time":time_stamp,"term_id": term_id, "address": address, "lat": lat, "long": lng, "num_bikes":num_bikes,"num_empty_docks":num_empty_docks, "obj_id":obj_id,"term_num":term_num}

    biker_data = pd.DataFrame(biker_data)

    with open('bike_data.csv', 'a') as f:
        biker_data.to_csv(f, header=False,index=False)


In [18]:
# This code creates a function that will continuously make the API call at a given interval
def periodic_work(interval):
    while True:
        #change this to the function you want to call, or paste in the code you want to run
        BikerData()
        #interval should be an integer, the number of seconds to wait
        time.sleep(interval)

In [ ]:
# Make the API call every 3 minutes (180 seconds)
periodic_work(180)